
<h2 id="Rough-volatility-with-Python">Rough volatility with Python<a class="anchor-link" href="#Rough-volatility-with-Python">¶</a></h2><p>
</p><p>Jim Gatheral</p>
<p>For Python Quants, New York, Friday May 6, 2016</p>
<h3><img align="left" src="http://mfe.baruch.cuny.edu/wp-content/uploads/2014/07/BaruchLogo2.png" width="160"/></h3><h3><img align="right" src="http://mfe.baruch.cuny.edu/wp-content/uploads/2014/07/MFElogo.png" width="100"/></h3>



<h3 id="Acknowledgements">Acknowledgements<a class="anchor-link" href="#Acknowledgements">¶</a></h3><p>The code in this iPython notebook used to be in R.  I am very grateful to Yves Hilpisch and Michael Schwed for translating my R-code to Python.</p>
<p>For slideshow functionality I use RISE by Damián Avila.</p>
$$
\newcommand{\beas}{\begin{eqnarray*}}
\newcommand{\eeas}{\end{eqnarray*}}
\newcommand{\bea}{\begin{eqnarray}}
\newcommand{\eea}{\end{eqnarray}}
\newcommand{\ben}{\begin{enumerate}}
\newcommand{\een}{\end{enumerate}}
\newcommand{\bi}{\begin{itemize}}
\newcommand{\ei}{\end{itemize}}
\newcommand{\beq}{\begin{equation}}
\newcommand{\eeq}{\end{equation}}
\newcommand{\bv}{\begin{verbatim}}
\newcommand{\ev}{\end{verbatim}}
\newcommand{\E}{\mathbb{E}}
\newcommand{\R}{\mathbb{R}}
\newcommand{\mP}{\mathbb{P}}
\newcommand{\mQ}{\mathbb{Q}}
\newcommand{\sigl}{\sigma_L}
\newcommand{\BS}{\rm BS}
\newcommand{\vix}{\text{VIX}}
\newcommand{\p}{\partial}
\newcommand{\var}{{\rm var}}
\newcommand{\cov}{{\rm cov}}
\newcommand{\mt}{\mathbf{t}}
\newcommand{\mS}{\mathbf{S}}
\newcommand{\tC}{\widetilde{C}}
\newcommand{\hC}{\widehat{C}}
\newcommand{\cE}{\mathcal{E}}
\newcommand{\tH}{\widetilde{H}}
\newcommand{\cD}{\mathcal{D}}
\newcommand{\cM}{\mathcal{M}}
\newcommand{\cS}{\mathcal{S}}
\newcommand{\cR}{\mathcal{R}}
\newcommand{\cF}{\mathcal{F}}
\newcommand{\cV}{\mathcal{V}}
\newcommand{\cG}{\mathcal{G}}
\newcommand{\cv}{\mathcal{v}}
\newcommand{\cg}{\mathcal{g}}
\newcommand{\cL}{\mathcal{L}}
\newcommand{\cO}{\mathcal{O}}
\newcommand{\dt}{\Delta t}
\newcommand{\tr}{{\rm tr}}
\newcommand{\sgn}{\mathrm{sign}}
\newcommand{\ee}[1]{{\mathbb{E}\left[{#1}\right]}}
\newcommand{\eef}[1]{{\mathbb{E}\left[\left.{#1}\right|\cF_t\right]}}
\newcommand{\eefm}[2]{{\mathbb{E}^{#2}\left[\left.{#1}\right|\cF_t\right]}}
\newcommand{\angl}[1]{{\langle{#1}\rangle}}
$$



<h3 id="Outline-of-presentation">Outline of presentation<a class="anchor-link" href="#Outline-of-presentation">¶</a></h3><ul>
<li><p>The time series of historical volatility</p>
<ul>
<li>Scaling properties</li>
</ul>
</li>
<li><p>The RFSV model</p>
</li>
<li><p>Pricing under rough volatility</p>
</li>
<li><p>Forecasting realized variance</p>
</li>
<li><p>The time series of variance swaps</p>
</li>
<li><p>Relating historical and implied</p>
</li>
</ul>



<h3 id="The-time-series-of-realized-variance">The time series of realized variance<a class="anchor-link" href="#The-time-series-of-realized-variance">¶</a></h3><ul>
<li><p>Assuming an underlying variance process $v_s$, integrated variance $
\frac 1 \delta \,\int_t^{t+\delta}\,v_s\,ds$ may (in principle) be estimated arbitrarily accurately given enough price data.</p>
<ul>
<li>In practice, market microstructure noise makes estimation harder at very high frequency.</li>
<li>Sophisticated estimators of integrated variance have been developed to adjust for market microstructure noise.  See Gatheral and Oomen <sup class="reference" id="cite_ref-GO"><a href="#cite_note-GO"><span>[</span>6<span>]</span></a></sup> (for example) for details of these.</li>
</ul>
</li>
</ul>



<ul>
<li><p>The Oxford-Man Institute of Quantitative Finance makes historical realized variance (RV) estimates freely available at <a href="http://realized.oxford-man.ox.ac.uk">http://realized.oxford-man.ox.ac.uk</a>.  These estimates are updated daily.</p>
<ul>
<li>Each day, for 21 different indices, all trades and quotes are used to estimate realized (or integrated) variance over the trading day from open to close.</li>
</ul>
</li>
</ul>
<ul>
<li>Using daily RV estimates as proxies for instantaneous variance, we may investigate the time series properties of $v_t$ empirically.</li>
</ul>



<p>First load all necessary Python libraries.</p>


In [ ]:

import warnings; warnings.simplefilter('ignore')
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib.mlab import stineman_interp
import pandas as pd
import pandas.io.data as web
import requests
import zipfile as zi 
import StringIO as sio
from sklearn import datasets, linear_model
import scipy.special as scsp
import statsmodels.api as sm
import math
import seaborn as sns; sns.set()
%matplotlib inline




<p>Then update and save the latest Oxford-Man data.</p>


In [ ]:

url = 'http://realized.oxford-man.ox.ac.uk/media/1366/'
url += 'oxfordmanrealizedvolatilityindices.zip'
data = requests.get(url, stream=True).content
z = zi.ZipFile(sio.StringIO(data))
z.extractall()




<p>There are many different estimates of realized variance, all of them very similar.  We will use the realized kernel estimates denoted by ".rk".</p>


In [ ]:

df = pd.read_csv('OxfordManRealizedVolatilityIndices.csv', index_col=0, header=2 )
rv1 = pd.DataFrame(index=df.index)
for col in df.columns:
    if col[-3:] == '.rk':
        rv1[col] = df[col]
rv1.index = [dt.datetime.strptime(str(date), "%Y%m%d") for date in rv1.index.values]




<p>Let's plot SPX realized variance.</p>


In [ ]:

spx = pd.DataFrame(rv1['SPX2.rk'])
spx.plot(color='red', grid=True, title='SPX realized variance',
         figsize=(16, 9), ylim=(0,0.003));




<p>Figure 1: Oxford-Man KRV estimates of SPX realized variance from January 2000 to the current date.</p>


In [ ]:

spx.head()



In [ ]:

spx.tail()




<p>We can get SPX data from Yahoo using the <code>DataReader</code> function:</p>


In [ ]:

SPX = web.DataReader(name = '^GSPC',data_source = 'yahoo', start='2000-01-01')
SPX = SPX['Adj Close']
SPX.plot(title='SPX',figsize=(14, 8));




<h3 id="The-smoothness-of-the-volatility-process">The smoothness of the volatility process<a class="anchor-link" href="#The-smoothness-of-the-volatility-process">¶</a></h3><p>For $q\geq 0$, we define the $q$th sample moment of differences of log-volatility at a given lag $\Delta$.($\angl{\cdot}$ denotes the sample average):</p>
$$
m(q,\Delta)=\angl{\left|\log \sigma_{t+\Delta} -\log \sigma_{t} \right|^q}
$$<p>For example</p>
$$
m(2,\Delta)=\angl{\left(\log \sigma_{t+\Delta} -\log \sigma_{t} \right)^2}
$$<p>is just the sample variance of differences in log-volatility at the lag $\Delta$.</p>



<h3 id="Scaling-of-$m(q,\Delta)$-with-lag-$\Delta$">Scaling of $m(q,\Delta)$ with lag $\Delta$<a class="anchor-link" href="#Scaling-of-$m(q,\Delta)$-with-lag-$\Delta$">¶</a></h3>


In [ ]:

spx['sqrt']= np.sqrt(spx['SPX2.rk'])
spx['log_sqrt'] = np.log(spx['sqrt'])

def del_Raw(q, x): 
    return [np.mean(np.abs(spx['log_sqrt'] - spx['log_sqrt'].shift(lag)) ** q)
            for lag in x]



In [ ]:

plt.figure(figsize=(8, 8))
plt.xlabel('$log(\Delta)$')
plt.ylabel('$log\  m(q.\Delta)$')
plt.ylim=(-3, -.5)

zeta_q = list()
qVec = np.array([.5, 1, 1.5, 2, 3])
x = np.arange(1, 100)
for q in qVec:
    plt.plot(np.log(x), np.log(del_Raw(q, x)), 'o') 
    model = np.polyfit(np.log(x), np.log(del_Raw(q, x)), 1)
    plt.plot(np.log(x), np.log(x) * model[0] + model[1])
    zeta_q.append(model[0])
    
print zeta_q




<p>Figure 2: $\log m(q,\Delta)$ as a function of $\log \Delta$, SPX.</p>



<h3 id="Monofractal-scaling-result">Monofractal scaling result<a class="anchor-link" href="#Monofractal-scaling-result">¶</a></h3><ul>
<li>From the above log-log plot, we see that for each $q$, $m(q,\Delta) \propto \Delta ^{\zeta_q}$.</li>
</ul>
<ul>
<li>How does $\zeta_q$ scale with $q$?</li>
</ul>



<h3 id="Scaling-of-$\zeta_q$-with-$q$">Scaling of $\zeta_q$ with $q$<a class="anchor-link" href="#Scaling-of-$\zeta_q$-with-$q$">¶</a></h3>


In [ ]:

plt.figure(figsize=(8,8))
plt.xlabel('q')
plt.ylabel('$\zeta_{q}$')
plt.plot(qVec, zeta_q, 'or')

line = np.polyfit(qVec[:4], zeta_q[:4],1)
plt.plot(qVec, line[0] * qVec + line[1])
h_est= line[0]
print(h_est)




<p>Figure 3: Scaling of $\zeta_q$ with $q$.</p>



<p>We find the monofractal scaling relationship</p>
$$
\zeta_q = q\,H
$$<p>with $H \approx 0.13$.</p>
<ul>
<li>Note however that $H$ does vary over time, in a narrow range.</li>
</ul>
<ul>
<li>Note also that our estimate of $H$ is biased high because we proxied instantaneous variance $v_t$ with its average over each day $\frac 1T\,\int_0^T\,v_t\,dt$, where $T$ is one trading day.</li>
</ul>



<h3 id="Estimated-$H$-for-all-indices">Estimated $H$ for all indices<a class="anchor-link" href="#Estimated-$H$-for-all-indices">¶</a></h3><p>We now repeat this analysis for all 21 indices in the Oxford-Man dataset.</p>


In [ ]:

def dlsig2(sic, x, pr=False):
    if pr:
        a= np.array([(sig-sig.shift(lag)).dropna() for lag in x])
        a=a ** 2
        print a.info()
    return [np.mean((sig-sig.shift(lag)).dropna() ** 2) for lag in x]



In [ ]:

h = list()
nu = list()

for col in rv1.columns:
    sig = rv1[col]
    sig = np.log(np.sqrt(sig))
    sig = sig.dropna()
    model = np.polyfit(np.log(x), np.log(dlsig2(sig, x)), 1)
    nu.append(np.sqrt(np.exp(model[1])))
    h.append(model[0]/2.)
    
OxfordH = pd.DataFrame({'names':rv1.columns, 'h_est': h, 'nu_est': nu})
  



In [ ]:

OxfordH




<h3 id="Distributions-of-$(\log-\sigma_{t+\Delta}-\log-\sigma_t)$-for-various-lags-$\Delta$">Distributions of $(\log \sigma_{t+\Delta}-\log \sigma_t)$ for various lags $\Delta$<a class="anchor-link" href="#Distributions-of-$(\log-\sigma_{t+\Delta}-\log-\sigma_t)$-for-various-lags-$\Delta$">¶</a></h3><p>Having established these beautiful scaling results for the moments, how do the histograms look?</p>


In [ ]:

def plotScaling(j, scaleFactor):
    col_name = rv1.columns[j]
    v = rv1[col_name]
    x = np.arange(1,101)
    
    def xDel(x, lag):
        return x-x.shift(lag)
    
    def sdl(lag):
        return (xDel(np.log(v), lag)).std()
    
    sd1 = (xDel(np.log(v), 1)).std()
    h = OxfordH['h_est'][j]
    f, ax = plt.subplots(2,2,sharex=False, sharey=False, figsize=(10, 10))
    
    for i_0 in range(0, 2):
        for i_1 in range(0, 2):
            la = scaleFactor ** (i_1*1+i_0*2)
        
            hist_val = xDel(np.log(v), la).dropna()
            std = hist_val.std()
            mean = hist_val.mean()
        
            ax[i_0][i_1].set_title('Lag = %s Days' %la)
            n, bins, patches = ax[i_0][i_1].hist(hist_val.values, bins=100,
                                   normed=1, facecolor='green',alpha=0.2)
            ax[i_0][i_1].plot(bins, mlab.normpdf(bins,mean,std), "r")
            ax[i_0][i_1].plot(bins, mlab.normpdf(bins,0,sd1 * la ** h), "b--")
            hist_val.plot(kind='density', ax=ax[i_0][i_1])
        
   



In [ ]:

 plotScaling(1,5)




<p>Figure 4: Histograms of $(\log \sigma_{t+\Delta}-\log \sigma_t)$ for various lags $\Delta$; normal fit in red; $\Delta=1$ normal fit scaled by $\Delta^{0.14}$ in blue.</p>



<h3 id="Universality?">Universality?<a class="anchor-link" href="#Universality?">¶</a></h3><ul>
<li><span>[Gatheral, Jaisson and Rosenbaum]<sup class="reference" id="cite_ref-GJR"><a href="#cite_note-GJR"><span>[</span>5<span>]</span>&lt;/a&gt;&lt;/sup&gt; compute daily realized variance estimates over one hour windows for DAX and Bund futures contracts, finding similar scaling relationships.</a></sup></span></li>
</ul>
<ul>
<li><p>We have also checked that Gold and Crude Oil futures scale similarly.</p>
<ul>
<li>Although the increments $(\log \sigma_{t+\Delta}-\log \sigma_t)$ seem to be fatter tailed than Gaussian.  </li>
</ul>
</li>
</ul>



<h3 id="A-natural-model-of-realized-volatility">A natural model of realized volatility<a class="anchor-link" href="#A-natural-model-of-realized-volatility">¶</a></h3><ul>
<li><p>As noted originally by <span>[Andersen et al.]<sup class="reference" id="cite_ref-ABDE"><a href="#cite_note-ABDE"><span>[</span>1<span>]</span>&lt;/a&gt;&lt;/sup&gt;, distributions of differences in the log of realized volatility are close to Gaussian.</a></sup></span></p>
<ul>
<li>This motivates us to model $\sigma_t$ as a lognormal random variable.</li>
</ul>
</li>
</ul>
<ul>
<li>Moreover, the scaling property of variance of RV differences suggests the model:</li>
</ul>
<p><a name="eq:dataDriven"></a>(1)
$$
\log \sigma_{t+\Delta} - \log \sigma_t =\nu\,\left( W^H_{t+\Delta}-W^H_t\right)
$$</p>
<p>where $W^H$ is fractional Brownian motion.</p>



<ul>
<li>In  <span>[Gatheral, Jaisson and Rosenbaum]<sup class="reference" id="cite_ref-GJR"><a href="#cite_note-GJR"><span>[</span>5<span>]</span>&lt;/a&gt;&lt;/sup&gt;, we refer to a stationary version of </a><a href="#eq:dataDriven">(1)</a> as the RFSV (for Rough Fractional Stochastic Volatility) model.</sup></span></li>
</ul>



<h3 id="Fractional-Brownian-motion-(fBm)">Fractional Brownian motion (fBm)<a class="anchor-link" href="#Fractional-Brownian-motion-(fBm)">¶</a></h3><ul>
<li><p><em>Fractional Brownian motion</em> (fBm) $\{W^H_t; t \in \mathbb{R}\}$ is the unique Gaussian process with mean zero and autocovariance function
$$
\ee{ W^H_t\,W^H_s  } = \frac12\,\left\{ |t|^{2\,H}+|s|^{2\,H}-|t-s|^{2\,H}  \right\}
$$
where $H \in (0,1)$ is called the <em>Hurst index</em> or parameter.</p>
<ul>
<li><p>In particular, when $H=1/2$, fBm is just Brownian motion.</p>
</li>
<li><p>If $H&gt;1/2$, increments are positively correlated.% so the process is trending.</p>
</li>
<li>If $H&lt;1/2$, increments are negatively correlated.% so the process is reverting.</li>
</ul>
</li>
</ul>



<h3 id="Representations-of-fBm">Representations of fBm<a class="anchor-link" href="#Representations-of-fBm">¶</a></h3><p>There are infinitely many possible representations of fBm in terms of Brownian motion.  For example, with $\gamma = \frac 12 - H$,</p>
<blockquote><div style="background-color:#add8e6; color:#FFFFFF; font-style: normal;  "><h4>
Mandelbrot-Van Ness</h4>
</div>
<div style="background-color:#E8E8E8; color:#000000; font-style: normal; ">
<br/>

$$
W^H_t ={C_H}\,\left\{\int_{-\infty}^t \,\frac{dW_s}{(t-s)^\gamma} - \int_{-\infty}^0 \,\frac{dW_s}{(-s)^\gamma}\right\}.
$$
<br/>
</div>
</blockquote>



<p>The choice</p>
$$
C_H = \sqrt{ \frac{2\,H\,\Gamma(3/2-H)}{\Gamma(H+1/2)\,\Gamma(2-2\,H)}}
$$<p>ensures that</p>
$$
\ee{W^H_t\,W^H_s }= \frac{1}{2}\,\left\{t^{2 H} + s^{2 H} - |t-s|^{2 H}\right\}.
$$



<h3 id="Does-simulated-RSFV-data-look-real?">Does simulated RSFV data look real?<a class="anchor-link" href="#Does-simulated-RSFV-data-look-real?">¶</a></h3>



<h3><img align="left" src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/02/vol2.png" width="900"/></h3>



<p>Figure 8: Volatility of SPX (above) and of the RFSV model (below).</p>



<h3 id="Remarks-on-the-comparison">Remarks on the comparison<a class="anchor-link" href="#Remarks-on-the-comparison">¶</a></h3><ul>
<li><p>The simulated and actual graphs look very alike.</p>
</li>
<li><p>Persistent periods of high volatility alternate with low volatility periods.</p>
</li>
<li><p>$H \sim 0.1$ generates very rough looking sample paths (compared with $H=1/2$ for Brownian motion).</p>
</li>
<li><p>Hence <em>rough volatility</em>.</p>
</li>
</ul>



<ul>
<li><p>On closer inspection, we observe fractal-type behavior.</p>
</li>
<li><p>The graph of volatility over a small time period looks like the same graph over a much longer time period.</p>
</li>
<li><p>This feature of volatility has been investigated both empirically and theoretically in, for example, <span>[Bacry and Muzy]<sup class="reference" id="cite_ref-BacryMuzy"><a href="#cite_note-BacryMuzy"><span>[</span>3<span>]</span>&lt;/a&gt;&lt;/sup&gt;
.</a></sup></span></p>
</li>
<li><p>In particular, their Multifractal Random Walk (MRW) is related to a limiting case of the RSFV model as $H \to 0$.</p>
</li>
</ul>



<h3 id="Pricing-under-rough-volatility">Pricing under rough volatility<a class="anchor-link" href="#Pricing-under-rough-volatility">¶</a></h3><p>The foregoing behavior suggest the following model (see <span>[Bayer et al.]<sup class="reference" id="cite_ref-BayerFriz"><a href="#cite_note-BayerFriz"><span>[</span>2<span>]</span>&lt;/a&gt;&lt;/sup&gt; for volatility under the real (or historical or physical) measure $\mP$:</a></sup></span></p>
$$
 \log \sigma_t =\nu\,W^H_t.
$$<p>Let $\gamma=\frac{1}{2}-H$.  We choose the Mandelbrot-Van Ness representation of fractional Brownian motion $W^H$ as follows:</p>
$$
W^H_t ={C_H}\,\left\{\int_{-\infty}^t \,\frac{dW^{\mP}_s}{(t-s)^\gamma} - \int_{-\infty}^0 \,\frac{dW^{\mP}_s}{(-s)^\gamma}\right\}.
$$



<p>Then</p>
$$
\begin{eqnarray}
&amp;&amp;\log v_u - \log v_t \nonumber\\
&amp;=&amp;\nu\,C_H\,\left\{ \int_t^u\,\frac{1}{(u-s)^\gamma}\,d{W}^{\mP}_s  +\int_{-\infty}^t\,\left[ \frac{1}{(u-s)^\gamma}-\frac{1}{(t-s)^\gamma} \right]\,d{W}^{\mP}_s\right\}\nonumber\\
&amp;=:&amp; 2\,\nu\,C_H\,\left[M_t(u)+ Z_t(u)\right].
\end{eqnarray}
$$<ul>
<li><p>Note that $\eefm{M_t(u)}{\mP}=0$ and $Z_t(u)$ is $\cF_t$-measurable.</p>
<ul>
<li>To price options, it would seem that we would need to know $\cF_t$, the entire history of the Brownian motion $W_s$ for $s<t>
</t></li></ul>
</li>
</ul>



<h3 id="Pricing-under-$\mP$">Pricing under $\mP$<a class="anchor-link" href="#Pricing-under-$\mP$">¶</a></h3><p>Let</p>
$$
\tilde W^{\mP}_t(u) := \sqrt{2\,H}\,\int_t^u\,\frac{dW^{\mP}_s}{(u-s)^\gamma}
$$<p>With 
$\eta := 2\,\nu\,C_H/\sqrt{2\,H}$ we have $2\,\nu\,C_H\, M_t(u)
= \eta\, \tilde W^{\mP}_t(u)$ so denoting the stochastic exponential by $\cE(\cdot)$, we may write</p>
$$
\begin{eqnarray}
v_u &amp;=&amp; v_t \exp\left\{  \eta \tilde W^{\mP}_t(u) +
 2\,\nu\,C_H\, 
Z_t(u) \right\}\nonumber\\
&amp;=&amp; \eefm{v_u}{\mP}\,\cE \left(\eta\,\tilde W^{\mP}_t(u) \right).
%\label{eq:rBergomiP}
\end{eqnarray}
$$<p></p>



<ul>
<li>The conditional distribution of $v_u$ depends on $\cF_t$ only through the variance forecasts $\eefm{v_u}{\mP}$, </li>
</ul>
<ul>
<li>To price options, one does not need to know $\cF_t$, the entire history of the Brownian motion $W_s^{\mP}$ for $s<t>
</t></li></ul>



<h3 id="Pricing-under-$\mQ$">Pricing under $\mQ$<a class="anchor-link" href="#Pricing-under-$\mQ$">¶</a></h3><p>Our model under $\mP$ reads:</p>
<p><a name="eq:Pmodel"></a>(2)
$$
v_u =\eefm{v_u}{\mP}\,\cE\left(\eta\,\tilde W^{\mP}_t(u) \right).
%\label{eq:Pmodel}
$$</p>
<p>Consider some general change of measure</p>
$$
dW^{\mP}_s = dW^{\mQ}_s + \lambda_s\,ds,
%\label{eq:dQdP}
$$<p>where $\{ \lambda_s: s &gt; t \}$  has a natural interpretation as the price of volatility risk.</p>



<p>We may then rewrite <a href="#eq:Pmodel">(2)</a> as</p>
$$
v_u
=  \eefm{v_u}{\mP}\,\cE\left(\eta\,\tilde W^{\mQ}_t(u) \right)
\exp \left\{ \eta\,\sqrt{2\,H}\, \int_t^u\,\frac{\lambda_s}{(u-s)^\gamma}\,ds\right\}.
%\label{eq:explicitBergomiQ1}
$$<ul>
<li><p>Although the conditional distribution of $v_u$ under $\mP$ is lognormal, it will not be lognormal in general under $\mQ$.</p>
<ul>
<li>The upward sloping smile in VIX options means $\lambda_s$ cannot be deterministic in this picture.</li>
</ul>
</li>
</ul>



<h3 id="The-rough-Bergomi-(rBergomi)-model">The rough Bergomi (rBergomi) model<a class="anchor-link" href="#The-rough-Bergomi-(rBergomi)-model">¶</a></h3><p>Let's nevertheless consider the simplest change of measure</p>
$$
d{W}^{\mP}_s = d{W}^{\mQ}_s + \lambda(s)\,ds, 
$$<p>where $\lambda(s)$ is a deterministic function of $s$.  Then from <a href="#eq:Pmodel">(2)</a>, we would have</p>
$$
\begin{eqnarray}
v_u 
&amp;=&amp;  \eefm{v_u}{\mP}\,\cE\left(\eta\,\tilde W^{\mQ}_t(u) \right)
\exp \left\{ \eta\,\sqrt{2\,H}\,  \int_t^u\,\frac{1}{(u-s)^\gamma}\,\lambda(s)\,ds\right\}\nonumber\\
&amp;=&amp; \xi_t(u) \,\cE\left(\eta\,\tilde W^{\mQ}_t(u) \right)%\label{eq:explicitBergomiQ}
\end{eqnarray}
$$<p>where the forward variances $\xi_t(u) =  \eefm{v_u}{\mQ}$ are (at least in principle) tradable and  observed in the market.</p>



<ul>
<li><p>$\xi_t(u)$ is the product of two terms:</p>
<ul>
<li>$ \eefm{v_u}{\mP}$ which depends on the historical path $\{W_s, s<t brownian="" motion="" of="" the="">
<li>a term which depends on the price of risk $\lambda(s)$.</li>
</t></li></ul>
</li>
</ul>



<h3 id="Features-of-the-rough-Bergomi-model">Features of the rough Bergomi model<a class="anchor-link" href="#Features-of-the-rough-Bergomi-model">¶</a></h3><ul>
<li>The rBergomi model is a non-Markovian generalization of the Bergomi model:
$$
\eef{v_u}\neq \E[v_u|v_t].
$$<ul>
<li>The rBergomi model is Markovian in the (infinite-dimensional) state vector $\eefm{v_u}{\mQ}=\xi_t(u)$.</li>
</ul>
</li>
</ul>
<ul>
<li>We have achieved our aim from Session 1 of replacing the exponential kernels in the Bergomi model with a power-law kernel.  </li>
</ul>
<ul>
<li>We may therefore expect that the rBergomi model will generate a realistic term structure of ATM volatility skew.</li>
</ul>



<h3 id="Re-interpretation-of-the-conventional-Bergomi-model">Re-interpretation of the conventional Bergomi model<a class="anchor-link" href="#Re-interpretation-of-the-conventional-Bergomi-model">¶</a></h3><ul>
<li><p>A conventional $n$-factor Bergomi model is not self-consistent for an arbitrary choice of the initial forward variance curve $\xi_t(u)$.</p>
<ul>
<li>$\xi_t(u)=\eef{v_u}$ should be consistent with the assumed dynamics.</li>
</ul>
</li>
</ul>



<ul>
<li><p>Viewed from the perspective of the fractional Bergomi model however:</p>
<ul>
<li>The initial curve $\xi_t(u)$ reflects the history $\{W_s; s<t brownian="" driving="" motion="" of="" the="" time="" to="" up="">
<li>The exponential kernels in the exponent of the conventional Bergomi model approximate more realistic power-law kernels.</li>
</t></li></ul>
</li>
</ul>
<ul>
<li>The conventional two-factor Bergomi model is then justified in practice as a tractable Markovian engineering approximation to a more realistic fractional Bergomi model.</li>
</ul>



<h3 id="The-stock-price-process">The stock price process<a class="anchor-link" href="#The-stock-price-process">¶</a></h3><ul>
<li>The observed anticorrelation between price moves and volatility moves may be  modeled naturally by anticorrelating the Brownian motion $W$ that drives the volatility process with the Brownian motion driving the price process.  </li>
</ul>
<ul>
<li>Thus
$$
\frac{dS_t}{S_t}=\sqrt{v_t}\,dZ_t
$$
with
$$
dZ_t = \rho\,dW_t + \sqrt{1-\rho^2}\,dW^\perp_t
$$
where $\rho$ is the correlation between volatility moves and price moves.</li>
</ul>



<h3 id="Simulation-of-the--rBergomi-model">Simulation of the  rBergomi model<a class="anchor-link" href="#Simulation-of-the--rBergomi-model">¶</a></h3><p>We simulate the rBergomi model as follows:</p>
<ul>
<li>Construct the  joint covariance matrix for the Volterra process $\tilde
W$ and the Brownian motion $Z$ and compute its Cholesky decomposition.</li>
</ul>
<ul>
<li>For each time, generate iid normal random vectors {and
  multiply them by the lower-triangular matrix obtained by the Cholesky
  decomposition} to get a $m \times 2\,n$ matrix of paths of $\tilde W$     and $Z$ with the correct joint marginals.</li>
</ul>



<ul>
<li>With these paths held in memory, we may evaluate the expectation under $\mQ$ of any payoff of interest.</li>
</ul>
<ul>
<li><p>This procedure is very slow!</p>
<ul>
<li>Speeding up the simulation is work in progress.</li>
</ul>
</li>
</ul>



<h3 id="Guessing-rBergomi-model-parameters">Guessing rBergomi model parameters<a class="anchor-link" href="#Guessing-rBergomi-model-parameters">¶</a></h3><ul>
<li>The rBergomi model has only three parameters: $H$, $\eta$ and $\rho$.</li>
</ul>
<ul>
<li>If we had a fast simulation, we could just iterate on these parameters to find the best fit to observed option prices.  But we don't.</li>
</ul>



<ul>
<li><p>However, the model parameters $H$, $\eta$ and $\rho$ have very direct interpretations:</p>
<ul>
<li><p>$H$ controls the decay of ATM skew $\psi(\tau)$ for very short expirations.</p>
</li>
<li><p>The product $\rho\,\eta$ sets the level of the ATM skew for longer expirations.</p>
<ul>
<li>Keeping  $\rho\,\eta$ constant but decreasing $\rho$ (so as to make it more negative) pushes the minimum of each smile towards higher strikes. </li>
</ul>
</li>
</ul>
</li>
</ul>



<ul>
<li>So we can guess parameters in practice.</li>
</ul>
<ul>
<li>As we will see, even without proper calibration (<em>i.e.</em> just guessing parameters), rBergomi model fits to the volatility surface are amazingly good.</li>
</ul>



<h3 id="SPX-smiles-in-the-rBergomi-model">SPX smiles in the rBergomi model<a class="anchor-link" href="#SPX-smiles-in-the-rBergomi-model">¶</a></h3><ul>
<li><p>In Figures 9 and 10, we show how well a rBergomi model simulation with guessed parameters fits the SPX option market as of February 4, 2010, a day when the ATM volatility term structure happened to be pretty flat.</p>
<ul>
<li>rBergomi parameters were: $H=0.07$, $\eta=1.9$, $\rho=-0.9$.</li>
</ul>
</li>
</ul>
<ul>
<li>Only three parameters to get a very good fit to the whole SPX volatility surface!</li>
</ul>



<h3 id="rBergomi-fits-to-SPX-smiles-as-of-04-Feb-2010">rBergomi fits to SPX smiles as of 04-Feb-2010<a class="anchor-link" href="#rBergomi-fits-to-SPX-smiles-as-of-04-Feb-2010">¶</a></h3><h3><img align="left" src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/02/spxSmiles20140204-07.png" width="900"/></h3>



<p>Figure 9: Red and blue points represent bid and offer SPX implied volatilities; orange smiles are from the rBergomi simulation.</p>



<h3 id="Shortest-dated-smile-as-of-February-4,-2010">Shortest dated smile as of February 4, 2010<a class="anchor-link" href="#Shortest-dated-smile-as-of-February-4,-2010">¶</a></h3><h3><img align="left" src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/02/spxSmiles20140204-07-1.png" width="900"/></h3>



<p>Figure 10: Red and blue points represent bid and offer SPX implied volatilities; orange smile is from the rBergomi simulation.</p>



<h3 id="ATM-volatilities-and-skews">ATM volatilities and skews<a class="anchor-link" href="#ATM-volatilities-and-skews">¶</a></h3><p>In Figures 11 and 12, we see just how well the rBergomi model can match empirical skews and vols.  Recall also that the parameters we used are just guesses!</p>



<h3 id="Term-structure-of-ATM-skew-as-of-February-4,-2010">Term structure of ATM skew as of February 4, 2010<a class="anchor-link" href="#Term-structure-of-ATM-skew-as-of-February-4,-2010">¶</a></h3><h3><img align="left" src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/02/atmSkew20100204.png" width="900"/></h3>



<p>Figure 11: Blue points are empirical skews; the red line is from the rBergomi simulation.</p>



<h3 id="Term-structure-of-ATM-vol-as-of-February-4,-2010">Term structure of ATM vol as of February 4, 2010<a class="anchor-link" href="#Term-structure-of-ATM-vol-as-of-February-4,-2010">¶</a></h3><h3><img align="left" src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/02/atmVols20100204.png" width="900"/></h3>



<p>Figure 12: Blue points are empirical ATM volatilities; the red line is from the rBergomi simulation.</p>



<h3 id="Another-date">Another date<a class="anchor-link" href="#Another-date">¶</a></h3><ul>
<li>Now we take a look at another date: August 14, 2013, two days before the last expiration date in our dataset.</li>
</ul>
<ul>
<li>Options set at the open of August 16, 2013 so only one trading day left.</li>
</ul>
<ul>
<li>Note in particular that the extreme short-dated smile is well reproduced by the rBergomi model.</li>
</ul>
<ul>
<li>There is no need to add jumps!</li>
</ul>



<h3 id="SPX-smiles-as-of-August-14,-2013">SPX smiles as of August 14, 2013<a class="anchor-link" href="#SPX-smiles-as-of-August-14,-2013">¶</a></h3><h3><img align="left" src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/02/spxSmiles20130814-05.png" width="900"/></h3>



<p>Figure 13: Red and blue points represent bid and offer SPX implied volatilities; orange smiles are from the rBergomi simulation.</p>



<h3 id="The-forecast-formula">The forecast formula<a class="anchor-link" href="#The-forecast-formula">¶</a></h3><ul>
<li>In the RFSV model <a href="#eq:dataDriven">(1)</a>, $\log \sigma_t \approx \nu\,W^H_t+C$ for some constant $C$.</li>
</ul>
<ul>
<li><span>[Nuzman and Poor]<sup class="reference" id="cite_ref-NuzmanPoor"><a href="#cite_note-NuzmanPoor"><span>[</span>7<span>]</span>&lt;/a&gt;&lt;/sup&gt; show that $W^H_{t+\Delta}$ is conditionally Gaussian with conditional expectation</a></sup></span></li>
</ul>
$$\E[W^H_{t+\Delta}|\cF_t]=\frac{\cos(H\pi)}{\pi} \Delta^{H+1/2} \int_{-\infty}^t \frac{W^H_s}{(t-s+\Delta)(t-s)^{H+1/2}} ds
$$<p></p>
<p>and conditional variance</p>
$$
\text{Var}[W^H_{t+\Delta}|\cF_t]=c\,\Delta^{2H}.
$$<p>where $$
c = \frac{\Gamma(3/2-H)}{\Gamma(H+1/2)\,\Gamma(2-2H)}.
$$</p>



<h3 id="The-forecast-formula">The forecast formula<a class="anchor-link" href="#The-forecast-formula">¶</a></h3><p>Thus, we obtain</p>
<blockquote><div style="background-color:#add8e6; color:#FFFFFF; font-style: normal;  "><h4>
Variance forecast formula</h4>
</div>
<div style="background-color:#E8E8E8; color:#000000; font-style: normal; ">
<br/>
<a name="eq:vForecast"></a>(3)
$$
\eefm{v_{t+\Delta}}{\mP}=\exp\left\{\eefm{\log(v_{t+\Delta})}{\mP}+2\, c\,\nu^2\Delta^{2\,H}\right\}
%\label{eq:vForecast}
$$

<br/>
</div>
</blockquote><p>where</p>
$$
\beas
&amp;&amp;\eefm{\log v_{t+\Delta}}{\mP}\\
&amp;&amp;= \frac{\cos(H\pi)}{\pi} \Delta^{H+1/2} \int_{-\infty}^t \frac{\log v_s}{(t-s+\Delta)(t-s)^{H+1/2}} ds.
\eeas
$$



<h3 id="Implement-variance-forecast-in-Python">Implement variance forecast in Python<a class="anchor-link" href="#Implement-variance-forecast-in-Python">¶</a></h3>


In [ ]:

def c_tilde(h):
    return scsp.gamma(3. / 2. - h) / scsp.gamma(h + 1. / 2.) * scsp.gamma(2. - 2. * h)

def forecast_XTS(rvdata, h, date, nLags, delta, nu):
    i = np.arange(nLags)
    cf = 1./((i + 1. / 2.) ** (h + 1. / 2.) * (i + 1. / 2. + delta))
    ldata = rvdata.truncate(after=date)
    l = len(ldata)
    ldata = np.log(ldata.iloc[l - nLags:])
    ldata['cf'] = np.fliplr([cf])[0]
    # print ldata
    ldata = ldata.dropna()
    fcst = (ldata.iloc[:, 0] * ldata['cf']).sum() / sum(ldata['cf'])
    
    return math.exp(fcst + 2 * nu ** 2 * c_tilde(h) * delta ** (2 * h))




<h3 id="SPX-actual-vs-forecast-variance">SPX actual vs forecast variance<a class="anchor-link" href="#SPX-actual-vs-forecast-variance">¶</a></h3>


In [ ]:

rvdata = pd.DataFrame(rv1['SPX2.rk'])
nu  = OxfordH['nu_est'][0] # Vol of vol estimate for SPX
h = OxfordH['h_est'][0] 
n = len(rvdata)
delta = 1
nLags = 500
dates = rvdata.iloc[nLags:n-delta].index
rv_predict = [forecast_XTS(rvdata, h=h, date=d, nLags=nLags,
                           delta=delta, nu=nu) for d in dates]
rv_actual = rvdata.iloc[nLags+delta:n].values




<h3 id="Scatter-plot-of-delta-days-ahead-predictions">Scatter plot of delta days ahead predictions<a class="anchor-link" href="#Scatter-plot-of-delta-days-ahead-predictions">¶</a></h3>


In [ ]:

plt.figure(figsize=(8, 8))
plt.plot(rv_predict, rv_actual, 'bo');




<p>Figure 14: Actual vols vs predicted vols.</p>



<h3 id="Superimpose-actual-and-predicted-vols">Superimpose actual and predicted vols<a class="anchor-link" href="#Superimpose-actual-and-predicted-vols">¶</a></h3>


In [ ]:

plt.figure(figsize=(11, 6))
vol_actual = np.sqrt(np.multiply(rv_actual,252))
vol_predict = np.sqrt(np.multiply(rv_predict,252))
plt.plot(vol_actual, "b")
plt.plot(vol_predict, "r");




<p>Figure 15: Actual volatilities in blue; predicted vols in red.</p>



<h3 id="Forecasting-the-variance-swap-curve">Forecasting the variance swap curve<a class="anchor-link" href="#Forecasting-the-variance-swap-curve">¶</a></h3><p>Finally, we forecast the whole variance swap curve using the variance forecasting formula <a href="#eq:vForecast">(3)</a>.</p>


In [ ]:

def xi(date, tt, nu,h, tscale):  # dt=(u-t) is in units of years
    rvdata = pd.DataFrame(rv1['SPX2.rk'])
    return [ forecast_XTS(rvdata,h=h,date=date,nLags=500,delta=dt*tscale,nu=nu) for dt in tt]

nu = OxfordH["nu_est"][0]
h = OxfordH["h_est"][0]

def varSwapCurve(date, bigT, nSteps, nu, h, tscale, onFactor):
  # Make vector of fwd variances
  tt = [ float(i) * (bigT) / nSteps for i in range(nSteps+1)]
  delta_t = tt[1]
  xicurve = xi(date, tt, nu, h, tscale)
  xicurve_mid = (np.array(xicurve[0:nSteps]) + np.array(xicurve[1:nSteps+1])) / 2
  xicurve_int = np.cumsum(xicurve_mid) * delta_t
  varcurve1 = np.divide(xicurve_int, np.array(tt[1:]))
  varcurve = np.array([xicurve[0],]+list(varcurve1))
  varcurve = varcurve * onFactor * tscale #  onFactor is to compensate for overnight moves
  res = pd.DataFrame({"texp":np.array(tt), "vsQuote":np.sqrt(varcurve)})
  return(res)



In [ ]:

def varSwapForecast(date,tau,nu,h,tscale,onFactor):
  vsc = varSwapCurve(date, bigT=2.5, nSteps=100, nu=nu, h=h,
                    tscale=tscale, onFactor=onFactor) # Creates the whole curve
  x = vsc['texp']
  y = vsc['vsQuote']
  res = stineman_interp(tau,x,y,None)

  return(res)

# Test the function

tau = (.25,.5,1,2)
date = dt.datetime(2008,9,8)
varSwapForecast(date,tau,nu=nu,h=h,tscale=252,onFactor=1)




<h3 id="'Constructing-a-time-series-of-variance-swap-curves">'Constructing a time series of variance swap curves<a class="anchor-link" href="#'Constructing-a-time-series-of-variance-swap-curves">¶</a></h3><p>For each of 2,658 days from Jan 27, 2003 to August 31, 2013:</p>
<ul>
<li>We compute proxy variance swaps from closing prices of SPX options sourced from OptionMetrics (www.optionmetrics.com) via WRDS.</li>
</ul>
<ul>
<li>We form the forecasts $\eefm{v_u}{\mP}$ using <a href="#eq:vForecast">(3)</a> with 500 lags of SPX RV data sourced from The Oxford-Man Institute of Quantitative Finance (<a href="http://realized.oxford-man.ox.ac.uk">http://realized.oxford-man.ox.ac.uk</a>).</li>
</ul>



<ul>
<li>We note that the actual variance swap curve is a factor (of roughly 1.4) higher than the forecast, which we may attribute to a combination of overnight movements of the index and the price of volatility risk.</li>
</ul>
<ul>
<li>Forecasts must therefore be rescaled to obtain close-to-close realized variance forecasts.</li>
</ul>



<h3 id="3-month-forecast-vs-actual-variance-swaps">3-month forecast vs actual variance swaps<a class="anchor-link" href="#3-month-forecast-vs-actual-variance-swaps">¶</a></h3><h3><img align="left" src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/02/vsfa3m.png" width="900"/></h3>



<p>Figure 16: Actual (proxy) 3-month variance swap quotes in blue vs forecast in red (with no scaling factor).</p>



<h3 id="Ratio-of-actual-to-forecast">Ratio of actual to forecast<a class="anchor-link" href="#Ratio-of-actual-to-forecast">¶</a></h3><h3><img align="left" src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/02/3mratio.png" width="900"/></h3>



<p>Figure 17: The ratio between 3-month actual variance swap quotes and 3-month forecasts.</p>



<h3 id="The-Lehman-weekend">The Lehman weekend<a class="anchor-link" href="#The-Lehman-weekend">¶</a></h3><ul>
<li><p>Empirically, it seems that the variance curve is a simple scaling factor times the forecast, but that this scaling factor is time-varying.</p>
<ul>
<li>We can think of this factor as having two multiplicative components: the overnight factor, and the price of volatility risk.</li>
</ul>
</li>
</ul>
<ul>
<li>Recall that as of the close on Friday September 12, 2008, it was widely believed that Lehman Brothers would be rescued over the weekend. By Monday morning, we knew that Lehman had failed. </li>
</ul>



<ul>
<li>In Figure 18, we see that variance swap curves just before and just after the collapse of Lehman are just rescaled versions of the RFSV forecast curves.</li>
</ul>



<h3 id="We-need-variance-swap-estimates-for-12-Sep-2008-and-15-Sep-2008">We need variance swap estimates for 12-Sep-2008 and 15-Sep-2008<a class="anchor-link" href="#We-need-variance-swap-estimates-for-12-Sep-2008-and-15-Sep-2008">¶</a></h3><p>We proxy these by taking SVI fits for the two dates and computing the log-strips.</p>


In [ ]:

varSwaps12 =(
    0.2872021, 0.2754535, 0.2601864, 0.2544684, 0.2513854, 0.2515314,
    0.2508418, 0.2520099, 0.2502763, 0.2503309, 0.2580933, 0.2588361, 
    0.2565093)

texp12 = (
    0.01916496, 0.04654346, 0.09582478, 0.19164956, 0.26830938, 0.29842574,
    0.51745380, 0.54483231, 0.76659822, 0.79397673, 1.26488706, 1.76317591, 
    2.26146475)

varSwaps15 = (
    0.4410505, 0.3485560, 0.3083603, 0.2944378, 0.2756881, 0.2747838, 
    0.2682212, 0.2679770, 0.2668113, 0.2706713, 0.2729533, 0.2689598, 
    0.2733176)

texp15 = (
    0.01095140, 0.03832991, 0.08761123, 0.18343600, 0.26009582, 0.29021218, 
    0.50924025, 0.53661875, 0.75838467, 0.78576318, 1.25667351, 1.75496235, 
    2.25325120)




<h3 id="Actual-vs-predicted-over-the-Lehman-weekend">Actual vs predicted over the Lehman weekend<a class="anchor-link" href="#Actual-vs-predicted-over-the-Lehman-weekend">¶</a></h3>


In [ ]:

nu = OxfordH['nu_est'][0]
h = OxfordH['h_est'][0]
date1 = dt.datetime(2008, 9, 12)
date2 = dt.datetime(2008, 9, 15)

# Variance curve fV model forecasts
tau1000 = [ float(i) * 2.5 / 1000. for i in range(1,1001)]
vs1 = varSwapForecast(date1, tau1000, nu=nu,h=h, tscale=252, onFactor=1.29)
vs2 = varSwapForecast(date2, tau1000, nu=nu,h=h, tscale=252, onFactor=1.29)



In [ ]:

plt.figure(figsize=(11, 6))
plt.plot(texp12, varSwaps12, "r")
plt.plot(texp15, varSwaps15, "b")
plt.plot(tau1000, vs1, "r--")
plt.plot(tau1000, vs2, "b--");




<p>Figure 18: SPX variance swap curves as of September 12, 2008 (red) and September 15, 2008 (blue). The dashed curves are RFSV model forecasts rescaled by the 3-month ratio ($1.29$) as of the Friday close.</p>



<h3 id="Remarks">Remarks<a class="anchor-link" href="#Remarks">¶</a></h3><p>We note that</p>
<ul>
<li>The actual variance swaps curves are very close to the forecast curves, up to a scaling factor.</li>
</ul>
<ul>
<li>We are able to explain the change in the variance swap curve with only one extra observation: daily variance over the trading day on Monday 15-Sep-2008. </li>
</ul>
<ul>
<li>The SPX options market appears to be backward-looking in a very sophisticated way.</li>
</ul>



<h3 id="The-Flash-Crash">The Flash Crash<a class="anchor-link" href="#The-Flash-Crash">¶</a></h3><ul>
<li>The so-called Flash Crash of Thursday May 6, 2010 caused  intraday realized variance to be much higher than normal.   </li>
</ul>
<ul>
<li>In Figure 19, we plot the actual variance swap curves as of the Wednesday and Friday market closes together with forecast curves rescaled by the 3-month ratio as of the close on Wednesday May 5 (which was $2.52$).  </li>
</ul>
<ul>
<li>We see that the actual variance curve as of the close on Friday is consistent with a forecast from the time series of realized variance that <em>includes</em> the anomalous price action of Thursday May 6. </li>
</ul>



<h3 id="Variance-swap-estimates">Variance swap estimates<a class="anchor-link" href="#Variance-swap-estimates">¶</a></h3><p>We again proxy variance swaps for 05-May-2010, 07-May-2010 and 10-May-2010 by taking SVI fits (see <span>[Gatheral and Jacquier]<sup class="reference" id="cite_ref-GatheralJacquierSSVI"><a href="#cite_note-GatheralJacquierSSVI"><span>[</span>4<span>]</span>&lt;/a&gt;&lt;/sup&gt; ) for the three dates and computing the log-strips.</a></sup></span></p>


In [ ]:

varSwaps5 = (
    0.4250369, 0.2552473, 0.2492892, 0.2564899, 0.2612677, 0.2659618, 0.2705928, 0.2761203,
    0.2828139, 0.2841165, 0.2884955, 0.2895839, 0.2927817, 0.2992602, 0.3116500)

texp5 = (
    0.002737851, 0.043805613, 0.120465435, 0.150581793, 0.197125257, 0.292950034,
    0.369609856, 0.402464066, 0.618754278, 0.654346338, 0.867898700, 0.900752909,
    1.117043121, 1.615331964, 2.631074606)
 
varSwaps7 = (
    0.5469727, 0.4641713, 0.3963352, 0.3888213, 0.3762354, 0.3666858, 0.3615814, 0.3627013,
    0.3563324, 0.3573946, 0.3495730, 0.3533829, 0.3521515, 0.3506186, 0.3594066)

texp7 = (
    0.01642710, 0.03832991, 0.11498973, 0.14510609, 0.19164956, 0.28747433, 0.36413415,
    0.39698836, 0.61327858, 0.64887064, 0.86242300, 0.89527721, 1.11156742, 1.60985626,
    2.62559890)

varSwaps10 = (
    0.3718439, 0.3023223, 0.2844810, 0.2869835, 0.2886912, 0.2905637, 0.2957070, 0.2960737,
    0.3005086, 0.3031188, 0.3058492, 0.3065815, 0.3072041, 0.3122905, 0.3299425)

texp10 = (
    0.008213552, 0.030116359, 0.106776181, 0.136892539, 0.183436003, 0.279260780,
    0.355920602, 0.388774812, 0.605065024, 0.640657084, 0.854209446, 0.887063655,
    1.103353867, 1.601642710, 2.617385352)



In [ ]:

date1 = dt.datetime(2010, 5, 5)
date2 = dt.datetime(2010, 5, 7)

vsf5 = varSwapCurve(date1, bigT=2.5, nSteps=100, nu=nu, h=h, tscale=252, onFactor=2.52)
vsf7 = varSwapCurve(date2, bigT=2.5, nSteps=100, nu=nu, h=h, tscale=252, onFactor=2.52)



In [ ]:

plt.figure(figsize=(11, 6))
plt.plot(texp5, varSwaps5, "r", label='May 5')
plt.plot(texp7, varSwaps7, "g", label='May 7')

plt.legend()
plt.xlabel("Time to maturity")
plt.ylabel("Variance swap quote")

plt.plot(vsf5['texp'], vsf5['vsQuote'], "r--")
plt.plot(vsf7['texp'], vsf7['vsQuote'], "g--");




<p>Figure 19: SPX variance swap curves as of May 5, 2010 (red) and May 7, 2010 (green). The dashed curves are RFSV model forecasts rescaled by the 3-month ratio ($2.52$) as of the close on Wednesday May 5.  The curve as of the close on May 7 is consistent with the forecast <strong>including</strong> the crazy moves on May 6.</p>



<h3 id="The-weekend-after-the-Flash-Crash">The weekend after the Flash Crash<a class="anchor-link" href="#The-weekend-after-the-Flash-Crash">¶</a></h3><p>Now we plot forecast and actual variance swap curves as of the close on Friday May 7 and Monday May 10.</p>


In [ ]:

date1 = dt.datetime(2010,5,7)
date2 = dt.datetime(2010,5,10)

vsf7 = varSwapCurve(date1, bigT=2.5, nSteps=100, nu=nu, h=h, tscale=252, onFactor=2.52)
vsf10 = varSwapCurve(date2, bigT=2.5, nSteps=100, nu=nu, h=h, tscale=252, onFactor=2.52)



In [ ]:

plt.figure(figsize=(11, 6))
plt.plot(texp7, varSwaps7, "g", label='May 7')
plt.plot(texp10, varSwaps10, "m", label='May 10')

plt.legend()
plt.xlabel("Time to maturity")
plt.ylabel("Variance swap quote")

plt.plot(vsf7['texp'], vsf7['vsQuote'], "g--")
plt.plot(vsf10['texp'], vsf10['vsQuote'], "m--");




<p>Figure 20: The May 10 actual curve is  inconsistent with a forecast that includes the Flash Crash.</p>



<p>Now let's see what happens if we exclude the Flash Crash from the time series used to generate the variance curve forecast.</p>


In [ ]:

plt.figure(figsize=(11, 6))
ax = plt.subplot(111)
rvdata_p = rvdata.drop((dt.datetime(2010, 5, 6)), axis=0)
rvdata.loc["2010-05-04":"2010-05-10"].plot(ax=ax, legend=False)
rvdata_p.loc["2010-05-04":"2010-05-10"].plot(ax=ax, legend=False);




<p>Figure 21: <code>rvdata_p</code> has the May 6 realized variance datapoint eliminated (green line).  Notice the crazy realized variance estimate for May 6!</p>
<p>We need a new variance curve forecast function that uses the new time series.</p>


In [ ]:

def xip(date, tt, nu,h, tscale):  # dt=(u-t) is in units of years
    rvdata = pd.DataFrame(rv1['SPX2.rk'])
    rvdata_p = rvdata.drop((dt.datetime(2010, 5, 6)), axis=0)
    return [ forecast_XTS(rvdata_p, h=h, date=date,nLags=500,
                          delta=delta_t * tscale, nu=nu) for delta_t in tt]

nu = OxfordH["nu_est"][0]
h = OxfordH["h_est"][0]

def varSwapCurve_p(date, bigT, nSteps, nu, h, tscale, onFactor):
  # Make vector of fwd variances
  tt = [ float(i) * (bigT) / nSteps for i in range(nSteps+1)]
  delta_t = tt[1]
  xicurve = xip(date, tt, nu, h, tscale)
  xicurve_mid = (np.array(xicurve[0:nSteps]) + np.array(xicurve[1:nSteps + 1])) / 2
  xicurve_int = np.cumsum(xicurve_mid) * delta_t
  varcurve1 = np.divide(xicurve_int, np.array(tt[1:]))
  varcurve = np.array([xicurve[0],]+list(varcurve1))
  varcurve = varcurve * onFactor * tscale #  onFactor is to compensate for overnight moves
  res = pd.DataFrame({"texp":np.array(tt), "vsQuote":np.sqrt(varcurve)})
  return(res)

def varSwapForecast_p(date, tau, nu, h, tscale, onFactor):
  vsc = varSwapCurve_p(date, bigT=2.5, nSteps=100, nu=nu, h=h,
                    tscale=tscale, onFactor=onFactor) # Creates the whole curve
  x = vsc['texp']
  y = vsc['vsQuote']
  res = stineman_interp(tau, x, y, None)

  return(res)

# Test the function

tau = (.25, .5 ,1, 2)
date = dt.datetime(2010, 5, 10)
varSwapForecast_p(date, tau, nu=nu, h=h, tscale=252, onFactor=1. / (1 - .35))




<p>Finally, we compare our new forecast curves with the actuals.</p>


In [ ]:

date1 = dt.datetime(2010, 5, 7)
date2 = dt.datetime(2010, 5, 10)

vsf7 = varSwapCurve(date1, bigT=2.5, nSteps=100, nu=nu, h=h, tscale=252, onFactor=2.52)
vsf10p = varSwapCurve_p(date2, bigT=2.5, nSteps=100, nu=nu, h=h, tscale=252, onFactor=2.52)



In [ ]:

plt.figure(figsize=(11, 6))
plt.plot(texp7, varSwaps7, "g", label='May 7')
plt.plot(texp10, varSwaps10, "m", label='May 10')

plt.legend()
plt.xlabel("Time to maturity")
plt.ylabel("Variance swap quote")

plt.plot(vsf7['texp'], vsf7['vsQuote'], "g--")
plt.plot(vsf10p['texp'], vsf10p['vsQuote'], "m--");




<p>Figure 22: The May 10 actual curve is consistent with a forecast that excludes the Flash Crash.</p>



<h3 id="Resetting-of-expectations-over-the-weekend">Resetting of expectations over the weekend<a class="anchor-link" href="#Resetting-of-expectations-over-the-weekend">¶</a></h3><ul>
<li>In Figures 20 and 22, we see that the actual variance swap curve on Monday, May 10  is consistent with a  forecast that excludes the  Flash Crash.</li>
</ul>
<ul>
<li>Volatility traders realized that the Flash Crash should not influence future realized variance projections.</li>
</ul>



<h3 id="Summary">Summary<a class="anchor-link" href="#Summary">¶</a></h3><ul>
<li><p>We uncovered a remarkable monofractal scaling relationship in historical volatility.</p>
<ul>
<li>A corollary is that volatility is not a long memory process, as widely believed.</li>
</ul>
</li>
</ul>
<ul>
<li>This leads to a natural non-Markovian stochastic volatility model under $\mP$.</li>
</ul>
<ul>
<li>The simplest specification of $\frac{d\mQ}{d\mP}$ gives a non-Markovian generalization of the Bergomi model.</li>
</ul>



<ul>
<li>The history of the Brownian motion $\lbrace W_s, s<t curve="" encoded="" for="" forward="" in="" is="" market.="" observed="" pricing="" required="" the="" variance="" which="">
</t></li></ul>
<ul>
<li>This model fits the observed volatility surface surprisingly well with very few parameters.</li>
</ul>
<ul>
<li>For perhaps the first time, we have a simple consistent model of historical and implied volatility.</li>
</ul>



<h2 id="References">References<a class="anchor-link" href="#References">¶</a></h2><p><br/></p>
<div class="reflist" style="list-style-type: decimal;">
<ol>
<li id="cite_note-ABDE"><span class="mw-cite-backlink"><b><a href="#cite_ref-ABDE">^</a></b></span>
Torben G Andersen, Tim Bollerslev, Francis X Diebold, and Heiko Ebens, The distribution of realized stock return volatility, *Journal of Financial Economics* **61**(1) 43-76 (2001).
</li>
<li id="cite_note-BayerFriz"><span class="mw-cite-backlink"><b><a href="#cite_ref-BayerFriz">^</a></b></span> 
Christian Bayer, Peter Friz and Jim Gatheral, Pricing under rough volatility, *Quantitative Finance* forthcoming, available at http://papers.ssrn.com/sol3/papers.cfm?abstract_id=2554754, (2015).
</li>
<li id="cite_note-BacryMuzy"><span class="mw-cite-backlink"><b><a href="#cite_ref-BacryMuzy">^</a></b></span>
Emmanuel Bacry and Jean-François Muzy, Log-infinitely divisible multifractal processes, 
*Communications in Mathematical Physics* **236**(3) 449-475 (2003).</li>
<li id="cite_note-GatheralJacquierSSVI"><span class="mw-cite-backlink"><b><a href="#cite_ref-GatheralJacquierSSVI">^</a></b></span>  Jim Gatheral and Antoine Jacquier, Arbitrage-free SVI volatility
surfaces, <span>*Quantitative Finance*</span> <span>**14**</span>(1) 59-71 (2014).</li>
<li id="cite_note-GJR"><span class="mw-cite-backlink"><b><a href="#cite_ref-GJR">^</a></b></span> Jim Gatheral, Thibault Jaisson and Mathieu Rosenbaum, Volatility is rough, available at http://papers.ssrn.com/sol3/papers.cfm?abstract_id=2509457, (2014).</li>
<li id="cite_note-GO"><span class="mw-cite-backlink"><b><a href="#cite_ref-GO">^</a></b></span> Jim Gatheral and Roel Oomen, Zero-intelligence realized variance estimation, *Finance and Stochastics* **14**(2) 249-283 (2010).</li>
<li id="cite_note-NuzmanPoor"><span class="mw-cite-backlink"><b><a href="#cite_ref-NuzmanPoor">^</a></b></span> Carl J. Nuzman and H. Vincent Poor, Linear estimation of self-similar processes via Lamperti’s transformation, *Journal of Applied Probability* **37**(2) 429-452 (2000).</li>
</ol>
</div>
